# Fed-BioMed to train a federated SGD regressor model

## Data 


This tutorial shows how to deploy in Fed-BioMed to solve a federated regression problem with scikit-learn.

In this tutorial we are using the wrapper of Fed-BioMed for the SGD regressor (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html).
The goal of the notebook is to train a model on a realistic dataset of (synthetic) medical information mimicking the ADNI dataset (http://adni.loni.usc.edu/). 

## Creating nodes

To proceed with the tutorial, we create 3 clients with corresponding dataframes of clinical information in .csv format. Each client has 300 data points composed by several features corresponding to clinical and medical imaging informations. **The data is entirely synthetic and randomly sampled to mimick the variability of the real ADNI dataset**. The training partitions are availables at the following link:

https://drive.google.com/file/d/1R39Ir60oQi8ZnmHoPz5CoGCrVIglcO9l/view?usp=sharing

The federated task we aim at solve is to predict a clinical variable (the mini-mental state examination, MMSE) from a combination of demographic and imaging features. The regressors variables are the following features:

['SEX', 'AGE', 'PTEDUCAT', 'WholeBrain.bl', 'Ventricles.bl', 'Hippocampus.bl', 'MidTemp.bl', 'Entorhinal.bl']

and the target variable is:

['MMSE.bl']
    

To create the federated dataset, we follow the standard procedure for node creation/population of Fed-BioMed. 
After activating the fedbiomed network with the commands

`source ./scripts/fedbiomed_environment network`

and 

`./scripts/fedbiomed_run network`

we create a first node by using the commands

`source ./scripts/fedbiomed_environment node`

`./scripts/fedbiomed_run node start`

We then poulate the node with the data of first client:

`./scripts/fedbiomed_run node config conf.ini add`

Thn, we select option 1 (csv dataset) to add the .csv partition of client 1, by just picking the .csv of client 1. We use `adni` as tag to save the selected dataset. We can further check that the data has been added by executing `./scripts/fedbiomed_run node list`

Following the same procedure, we create the other two nodes with the datasets of client 2 and client 3 respectively. To do so, we add and launch a `Node`using others configuration files

## Fed-BioMed Researcher

We are now ready to start the reseracher enviroment with the command `source ./scripts/fedbiomed_environment researcher`, and open the Jupyter notebook with `./scripts/fedbiomed_run researcher start`. 

We can first query the network for the adni dataset. In this case, the nodes are sharing the respective partitions unsing the same tag `adni`:

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from fedbiomed.researcher.requests import Requests
req = Requests()
req.list(verbose=True)

The code for network and data loader of the sklearn SGDRegressor can now be deployed in Fed-BioMed.
We first import the necessary module `SGDSkLearnModel` from `fedbiomed`:

**__init__** : we add here the needed sklearn libraries
       
**training_data** : you must return here a tuple (data,targets) that must be of the same type of 
your method partial_fit parameters. 

We note that this model performs a common standardization across federated datasets by **centering with respect to the same parameters**.

In [ ]:
from fedbiomed.common.training_plans import FedSGDRegressor
from fedbiomed.common.data import DataManager

from fedbiomed.common.optimizers import Optimizer
from fedbiomed.common.optimizers.declearn import AdamModule, FedProxRegularizer


class SGDRegressorTrainingPlan(FedSGDRegressor):
    # Declares and return dependencies
    def init_dependencies(self):
        deps = ["from torchvision import datasets, transforms",
                "from declearn.optimizer import Optimizer",
                "from fedbiomed.common.optimizers.declearn import AdamModule",
                "from fedbiomed.common.optimizers.declearn import FedProxRegularizer"]
        return deps

    def training_data(self):
        dataset = pd.read_csv(self.dataset_path, delimiter=',')
        regressors_col = ['AGE', 'WholeBrain.bl',
                          'Ventricles.bl', 'Hippocampus.bl', 'MidTemp.bl', 'Entorhinal.bl']
        target_col = ['MMSE.bl']
        
        # mean and standard deviation for normalizing dataset
        # it has been computed over the whole dataset
        scaling_mean = np.array([72.3, 0.7, 0.0, 0.0, 0.0, 0.0])
        scaling_sd = np.array([7.3e+00, 5.0e-02, 1.1e-02, 1.0e-03, 2.0e-03, 1.0e-03])
        
        X = (dataset[regressors_col].values-scaling_mean)/scaling_sd
        y = dataset[target_col]
        return DataManager(dataset=X, target=y.values.ravel(),  shuffle=True)

    # Defines and return a declearn optimizer
    def init_optimizer(self, optimizer_args):
        return Optimizer(lrate=.1 ,modules=[AdamModule()], regularizers=[FedProxRegularizer()])

**model_args** is a dictionary containing your model arguments, in case of SGDRegressor this will be max_iter and tol. n_features is provided to correctly initialize the SGDRegressor coef_ array.

**training_args** is a dictionary with parameters related to Federated Learning. 

In [ ]:
from fedbiomed.common.metrics import MetricTypes
RANDOM_SEED = 1234


model_args = {
    'max_iter':2000,
    'tol': 1e-5,
    'eta0':0.05,
    'n_features': 6,
    'random_state': RANDOM_SEED
}

training_args = {
    'epochs': 5,
    'loader_args': { 'batch_size': 32, },
    'test_ratio':.3,
    'test_metric': MetricTypes.MEAN_SQUARE_ERROR,
    'test_on_local_updates': True,
    'test_on_global_updates': True
}

The experiment can be now defined, by providing the `adni` tag, and running the local training on nodes with model defined in `model_path`, standard `aggregator` (FedAvg) and `client_selection_strategy` (all nodes used). Federated learning is going to be perfomed through 10 optimization rounds.

In [ ]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['adni']

# Add more rounds for results with better accuracy
#
#rounds = 40
rounds = 5

# select nodes participating to this experiment
exp = Experiment(tags=tags,
                 model_args=model_args,
                 training_plan_class=SGDRegressorTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

In [ ]:
# start federated training
exp.run()

2023-09-06 15:40:39,948 fedbiomed INFO - TRAINING 
					 NODE_ID: node_0874f153-2bbb-4e2b-b1ce-cfd20858a636 
					 Round 6 Epoch: 1 | Iteration: 80/100 (80%) | Samples: 3840/4800
 					 Loss: 2.566728 
					 ---------
2023-09-06 15:40:40,622 fedbiomed INFO - TRAINING 
					 NODE_ID: node_0874f153-2bbb-4e2b-b1ce-cfd20858a636 
					 Round 6 Epoch: 1 | Iteration: 90/100 (90%) | Samples: 4320/4800
 					 Loss: 2.679407 
					 ---------
2023-09-06 15:40:41,152 fedbiomed INFO - TRAINING 
					 NODE_ID: node_0874f153-2bbb-4e2b-b1ce-cfd20858a636 
					 Round 6 Epoch: 1 | Iteration: 100/100 (100%) | Samples: 4800/4800
 					 Loss: 2.452188 
					 ---------
2023-09-06 15:40:41,295 fedbiomed INFO - INFO
					 NODE node_0874f153-2bbb-4e2b-b1ce-cfd20858a636
					 MESSAGE: results uploaded successfully 
-----------------------------------------------------------------
2023-09-06 15:40:41,300 fedbiomed INFO - WARNING
					 NODE node_0874f153-2bbb-4e2b-b1ce-cfd20858a636
					 MESSAGE: optimizer info b

2023-09-06 15:40:51,613 fedbiomed INFO - INFO
					 NODE node_f42cc93b-396e-44dd-9585-46ee2469b4ab
					 MESSAGE: Detected changes in the optimizer!
-----------------------------------------------------------------
2023-09-06 15:40:51,621 fedbiomed INFO - INFO
					 NODE node_f42cc93b-396e-44dd-9585-46ee2469b4ab
					 MESSAGE: Detected changes in the optimizer!
-----------------------------------------------------------------
2023-09-06 15:40:51,629 fedbiomed INFO - INFO
					 NODE node_f42cc93b-396e-44dd-9585-46ee2469b4ab
					 MESSAGE: State node_state_06816012-946f-4a0d-908e-255c776a11b7 loaded
-----------------------------------------------------------------
2023-09-06 15:40:51,638 fedbiomed INFO - WARNING
					 NODE node_f42cc93b-396e-44dd-9585-46ee2469b4ab
					 MESSAGE: Optimizer 2loaded state {'config': {'lrate': 0.1, 'w_decay': 0.0, 'regularizers': [], 'modules': [('adam', {'beta_1': 0.5, 'beta_2': 0.99, 'amsgrad': False, 'eps': 1e-07}), ('momentum', {'beta': 0.9, 'nesterov': 

2023-09-06 15:41:03,673 fedbiomed INFO - WARNING
					 NODE node_0874f153-2bbb-4e2b-b1ce-cfd20858a636
					 MESSAGE: optimizer info bfore aving {'config': {'lrate': 0.1, 'w_decay': 0.0, 'regularizers': [], 'modules': [('adam', {'beta_1': 0.5, 'beta_2': 0.99, 'amsgrad': False, 'eps': 1e-07}), ('momentum', {'beta': 0.9, 'nesterov': False})]}, 'states': {'modules': [('adam', {'steps': 200, 'vmax': None, 'momentum': {'state': TorchVector with 8 coefs:
    conv1.weight: float32 torch.Tensor with shape (32, 1, 3, 3)
    conv1.bias: float32 torch.Tensor with shape (32,)
    conv2.weight: float32 torch.Tensor with shape (64, 32, 3, 3)
    conv2.bias: float32 torch.Tensor with shape (64,)
    fc1.weight: float32 torch.Tensor with shape (128, 9216)
    fc1.bias: float32 torch.Tensor with shape (128,)
    fc2.weight: float32 torch.Tensor with shape (10, 128)
    fc2.bias: float32 torch.Tensor with shape (10,)}, 'velocity': {'state': TorchVector with 8 coefs:
    conv1.weight: float32 torch.Tensor

2023-09-06 15:45:19,571 fedbiomed INFO - ERROR
					 NODE test_logger_node_96fa4b06-a639-4adf-9e68-3e75613a03b4
					 MESSAGE: mqtt+console ERROR message
-----------------------------------------------------------------
2023-09-06 15:45:32,437 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Messaging mock_researcher__res_8b3a8eb2-3218-4373-b4bd-b3502aa88ccf_XXX successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7f8ff50a6fe0>
-----------------------------------------------------------------
2023-09-06 15:45:34,523 fedbiomed INFO - WARNING
					 NODE node_1234
					 MESSAGE: Target data seems to be a regression, metric ACCURACY might not be appropriate
-----------------------------------------------------------------
2023-09-06 15:45:34,524 fedbiomed INFO - WARNING
					 NODE node_1234
					 MESSAGE: Target data seems to be a regression, metric ACCURACY might not be appropriate
-----------------------------------------------

2023-09-06 15:45:34,688 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Actual/True values (y_true) has more than two levels, using multiclass `weighted` calculation for the metric PRECISION
-----------------------------------------------------------------
2023-09-06 15:45:34,701 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Actual/True values (y_true) has more than two levels, using multiclass `samples` calculation for the metric F1_SCORE
-----------------------------------------------------------------
2023-09-06 15:45:34,703 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Actual/True values (y_true) has more than two levels, using multiclass `samples` calculation for the metric RECALL
-----------------------------------------------------------------
2023-09-06 15:45:34,711 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Actual/True values (y_true) has more than two levels, using multiclass `samples` calculation for the metric PRECISION
-----

# Declearn Optimizers with Scikit learn Perceptron Classifier

In [ ]:
from fedbiomed.common.training_plans import FedPerceptron
from fedbiomed.common.data import DataManager
import numpy as np

from fedbiomed.common.optimizers import Optimizer
from fedbiomed.common.optimizers.declearn import AdamModule, FedProxRegularizer

class SkLearnClassifierTrainingPlan(FedPerceptron):
    def init_dependencies(self):
        """Define additional dependencies.
        
        In this case, we rely on torchvision functions for preprocessing the images.
        """
        return ["from torchvision import datasets, transforms",
                "from fedbiomed.common.optimizers import Optimizer",
                "from fedbiomed.common.optimizers.declearn import AdamModule, FedProxRegularizer",]

    def training_data(self):
        """Prepare data for training.
        
        This function loads a MNIST dataset from the node's filesystem, applies some
        preprocessing and converts the full dataset to a numpy array. 
        Finally, it returns a DataManager created with these numpy arrays.
        """
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        
        X_train = dataset.data.numpy()
        X_train = X_train.reshape(-1, 28*28)
        Y_train = dataset.targets.numpy()
        return DataManager(dataset=X_train, target=Y_train,  shuffle=False)
    
    # Defines and return a declearn optimizer
    def init_optimizer(self, optimizer_args):
        return Optimizer(lr=.1 ,modules=[AdamModule()], regularizers=[FedProxRegularizer()])

In [ ]:
model_args = {'n_features': 28*28,
              'n_classes' : 10,
              'eta0':1e-6,
              'random_state':1234,
              'alpha':0.1 }

training_args = {
    'epochs': 3, 
    'batch_maxnum': 20,  # can be used to debugging to limit the number of batches per epoch
    'optimizer_args': {
        "lr" : 1e-3
    },
#    'log_interval': 1,  # output a logging message every log_interval batches
    'loader_args': { 'batch_size': 4, }, 
}

In [ ]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 3

# select nodes participating in this experiment
exp = Experiment(tags=tags,
                 model_args=model_args,
                 training_plan_class=SkLearnClassifierTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)


In [ ]:
exp.run(increase=True)

##  Testing

Once the federated model is obtained, it is possible to test it locally on an independent testing partition.
The test dataset is available at this link:

https://drive.google.com/file/d/1zNUGp6TMn6WSKYVC8FQiQ9lJAUdasxk1/

In [ ]:
!pip install matplotlib
!pip install gdown

Download the testing dataset on the local temporary folder.

In [ ]:
import os
import gdown
import tempfile
import zipfile
import pandas as pd
import numpy as np

from fedbiomed.common.constants import ComponentType
from fedbiomed.researcher.environ import environ


resource = "https://drive.google.com/uc?id=19kxuI146WA2fhcOU2_AvF8dy-ppJkzW7"

tmpdir = tempfile.TemporaryDirectory(dir=environ['TMP_DIR'])
base_dir = tmpdir.name

test_file = os.path.join(base_dir, "test_data.zip")
gdown.download(resource, test_file, quiet=False)

zf = zipfile.ZipFile(test_file)

for file in zf.infolist():
    zf.extract(file, base_dir)

# loading testing dataset
test_data = pd.read_csv(os.path.join(base_dir,'adni_validation.csv'))

In [ ]:
from sklearn.linear_model import SGDRegressor
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

Here we extract the relevant regressors and target from the testing data 

In [ ]:
regressors_col = ['AGE', 'WholeBrain.bl', 'Ventricles.bl',
                  'Hippocampus.bl', 'MidTemp.bl', 'Entorhinal.bl']
target_col = ['MMSE.bl']
X_test = test_data[regressors_col].values
y_test = test_data[target_col].values

To inspect the model evolution across FL rounds, we export `exp.aggregated_params()` containing models parameters collected at the end of each round. The MSE (Mean Squarred Error) should be decreasing at each iteration with the federated parameters obtained at each round. 

In [ ]:
scaling_mean = np.array([72.3, 0.7, 0.0, 0.0, 0.0, 0.0])
scaling_sd = np.array([7.3e+00, 5.0e-02, 1.1e-02, 1.0e-03, 2.0e-03, 1.0e-03])

testing_error = []


# we create here several instances of SGDRegressor using same sklearn arguments
# we have used for Federated Learning training
fed_model = exp.training_plan().model()
regressor_args = {key: model_args[key] for key in model_args.keys() if key in fed_model.get_params().keys()}

for i in range(rounds):
    fed_model.coef_ = exp.aggregated_params()[i]['params']['coef_'].copy()
    fed_model.intercept_ = exp.aggregated_params()[i]['params']['intercept_'].copy()  
    mse = np.mean((fed_model.predict((X_test-scaling_mean)/scaling_sd) - y_test)**2)
    testing_error.append(mse)

plt.plot(testing_error)
plt.title('FL testing loss')
plt.xlabel('FL round')
plt.ylabel('testing loss (MSE)')

We finally inspect the predictions of the final federated model on the testing data.

In [ ]:
y_predicted = fed_model.predict((X_test-scaling_mean)/scaling_sd)
plt.scatter(y_predicted, y_test, label='model prediction')
plt.xlabel('predicted')
plt.ylabel('target')
plt.title('Federated model testing prediction')

first_diag = np.arange(np.min(y_test.flatten()),
                       np.max(y_test.flatten()+1))
plt.scatter(first_diag, first_diag, label='correct Target')
plt.legend()

In [ ]:
a = X_test / scaling_sd
a.shape

In [ ]:
X_test.shape

In [ ]:
X_test[:,1] / scaling_sd[1] - a[:,1]